In [1]:
# Numbeo website contains country name and quality of life index from 2012-2019
# For geojson chloropleth mapping, we need to join the Numbeo table with standard ISO country code
# This python file will scrape Numbeo and ISO websites and merge by standard ISO country code for our geojson world map

In [2]:
# Import dependencies
import pandas as pd
import numpy as np
import json
import pymongo
from flask import Flask, jsonify
from bson.json_util import dumps

In [3]:
# Fix Python SSL CERTIFICATE_VERIFY_FAILED
import os, ssl
if (not os.environ.get('PYTHONHTTPSVERIFY', '') and
    getattr(ssl, '_create_unverified_context', None)): 
    ssl._create_default_https_context = ssl._create_unverified_context

In [4]:
# Numbeo has Quality of Life index from years 2012 to 2019
# Create list of links to scrape
# Loop to create URL links from 2012-2018
# link=[]
# for x in range(2012,2019):
#     link.append('https://www.numbeo.com/quality-of-life/rankings_by_country.jsp?title='+str(x)+'-Q1')
# print(link)

In [5]:
# # Append year 2019 URL link
# link.append('https://www.numbeo.com/quality-of-life/rankings_by_country.jsp')
# print(link)

In [6]:
# # Scrape and read as pandas dataframe
# for index,element in enumerate(link):
#     print(element)

In [7]:
# numDf = []
# for index,element in enumerate(link):
#     print(index,element)
#     numDf.append("numDF"+str(index+2012))
#     numDf[index] = pd.read_html("'"+str(element)+"'")


In [8]:
# Read in Numbeo HTML as Pandas table
#2019
table = pd.read_html('https://www.numbeo.com/quality-of-life/rankings_by_country.jsp')
numDf2019 = table[2][["Country","Quality of Life Index"]].rename(columns={
    'Quality of Life Index':'QII_2019'})
numDf2019.count()

Country     71
QII_2019    71
dtype: int64

In [9]:
# 2012
table2012 = pd.read_html('https://www.numbeo.com/quality-of-life/rankings_by_country.jsp?title=2012-Q1')
numDf2012 = table2012[2][["Country","Quality of Life Index"]].rename(columns={
    'Quality of Life Index':'QII_2012'})
numDf2012.count()

Country     51
QII_2012    51
dtype: int64

In [10]:
# 2013
table2013 = pd.read_html('https://www.numbeo.com/quality-of-life/rankings_by_country.jsp?title=2013-Q1')
numDf2013 = table2013[2][["Country","Quality of Life Index"]].rename(columns={
    'Quality of Life Index':'QII_2013'})
numDf2013.count()

Country     67
QII_2013    67
dtype: int64

In [11]:
# 2014
table2014 = pd.read_html('https://www.numbeo.com/quality-of-life/rankings_by_country.jsp?title=2014')
numDf2014 = table2014[2][["Country","Quality of Life Index"]].rename(columns={
    'Quality of Life Index':'QII_2014'})
numDf2014.count()

Country     68
QII_2014    68
dtype: int64

In [12]:
# 2015
table2015 = pd.read_html('https://www.numbeo.com/quality-of-life/rankings_by_country.jsp?title=2015')
numDf2015 = table2015[2][["Country","Quality of Life Index"]].rename(columns={
    'Quality of Life Index':'QII_2015'})
numDf2015.count()

Country     86
QII_2015    86
dtype: int64

In [13]:
# 2016
table2016 = pd.read_html('https://www.numbeo.com/quality-of-life/rankings_by_country.jsp?title=2016')
numDf2016 = table2016[2][["Country","Quality of Life Index"]].rename(columns={
    'Quality of Life Index':'QII_2016'})
numDf2016.count()

Country     56
QII_2016    56
dtype: int64

In [14]:
# 2017
table2017 = pd.read_html('https://www.numbeo.com/quality-of-life/rankings_by_country.jsp?title=2017')
numDf2017 = table2017[2][["Country","Quality of Life Index"]].rename(columns={
    'Quality of Life Index':'QII_2017'})
numDf2017.count()

Country     67
QII_2017    67
dtype: int64

In [15]:
# 2018
table2018 = pd.read_html('https://www.numbeo.com/quality-of-life/rankings_by_country.jsp?title=2018')
numDf2018 = table2018[2][["Country","Quality of Life Index"]].rename(columns={
    'Quality of Life Index':'QII_2018'})
numDf2018.count()

Country     60
QII_2018    60
dtype: int64

In [16]:
merge1 = pd.merge(numDf2019, numDf2018, how="outer", on="Country")
merge2 = pd.merge(numDf2017,merge1, how="outer", on="Country")
merge3 = pd.merge(numDf2016,merge2, how="outer", on="Country")
merge4 = pd.merge(numDf2015,merge3, how="outer", on="Country")
merge5 = pd.merge(numDf2014,merge4, how="outer", on="Country")
merge6 = pd.merge(numDf2013,merge5, how="outer", on="Country")
merge_final = pd.merge(numDf2012,merge6, how="outer", on="Country")
merge_final.tail()

,Country,QII_2012,QII_2013,QII_2014,QII_2015,QII_2016,QII_2017,QII_2019,QII_2018
82,Nigeria,NaN,NaN,NaN,2.99,NaN,NaN,NaN,NaN
83,Cambodia,NaN,NaN,NaN,-4.40,NaN,NaN,NaN,NaN
84,Bangladesh,NaN,NaN,NaN,-5.36,NaN,NaN,NaN,NaN
85,Vietnam,NaN,NaN,NaN,-19.52,NaN,36.12,88.82,81.09
86,Iceland,NaN,NaN,NaN,NaN,NaN,NaN,187.79,NaN


In [17]:
# # populate rank starting with 1
# merge_final['Rank'] = [x for x in range(1,len(merge_final)+1)]
# merge_final.head()

In [18]:
# Read in ISO HTML as Pandas Table
table = pd.read_html('https://www.iban.com/country-codes')
isoDf = table[0]
isoDf.head()

,Country,Alpha-2 code,Alpha-3 code,Numeric
0,Afghanistan,AF,AFG,4
1,Åland Islands,AX,ALA,248
2,Albania,AL,ALB,8
3,Algeria,DZ,DZA,12
4,American Samoa,AS,ASM,16


In [19]:
# Rename ISO columns
isoDf = isoDf.rename(columns={
    'Alpha-2 code':'iso_a2',
    'Alpha-3 code':'iso_a3',
    'Numeric':'iso_n'
})
isoDf.head()

,Country,iso_a2,iso_a3,iso_n
0,Afghanistan,AF,AFG,4
1,Åland Islands,AX,ALA,248
2,Albania,AL,ALB,8
3,Algeria,DZ,DZA,12
4,American Samoa,AS,ASM,16


In [20]:
# Left Join Numbeo and ISO by Country
countryDf = merge_final.merge(isoDf, on='Country', how='left')
countryDf.head()

,Country,QII_2012,QII_2013,QII_2014,QII_2015,QII_2016,QII_2017,QII_2019,QII_2018,iso_a2,iso_a3,iso_n
0,Switzerland,194.11,215.71,206.23,222.94,208.54,173.54,195.93,190.81,CH,CHE,756.0
1,Germany,184.42,204.84,192.69,195.94,199.70,189.74,187.05,190.04,DE,DEU,276.0
2,Norway,183.43,173.86,168.41,175.09,188.90,165.93,181.86,176.27,NO,NOR,578.0
3,United Arab Emirates,177.07,186.01,173.27,153.68,144.09,139.27,167.81,163.54,NaN,NaN,NaN
4,New Zealand,174.28,163.17,168.56,175.51,201.06,184.74,185.58,182.40,NZ,NZL,554.0


In [21]:
# Check for null columns
null_columns = countryDf.columns[countryDf.isnull().any()]
countryDf[null_columns].isnull().sum()

QII_2012    36
QII_2013    20
QII_2014    19
QII_2015     1
QII_2016    31
QII_2017    20
QII_2019    16
QII_2018    27
iso_a2      17
iso_a3      17
iso_n       17
dtype: int64

In [22]:
# Find 17 countries that did not match in the join
countryDf[countryDf.iso_a2.isnull()].Country

3       United Arab Emirates
10               Netherlands
11             United States
17            United Kingdom
22            Czech Republic
37    Bosnia And Herzegovina
45               Philippines
46                    Russia
49                      Iran
54               South Korea
55                    Taiwan
66                 Venezuela
77        Dominican Republic
78                   Moldova
79                 Macedonia
81                   Bolivia
85                   Vietnam
Name: Country, dtype: object

In [23]:
# Clean each country one by one
# Match as Numbeo country

# Netherlands
isoDf[isoDf['Country'].str.contains('Neth', case=False)]

,Country,iso_a2,iso_a3,iso_n
157,Netherlands (the),NL,NLD,528


In [24]:
isoDf.loc[isoDf['Country'].str.contains('Neth', case=False), 'Country'] = 'Netherlands'
isoDf[isoDf['Country'].str.contains('Neth', case=False)]

,Country,iso_a2,iso_a3,iso_n
157,Netherlands,NL,NLD,528


In [25]:
# United States and United Kingdom and United Arab Emirates
isoDf[isoDf['Country'].str.contains('United', case=False)]

,Country,iso_a2,iso_a3,iso_n
219,"Tanzania, United Republic of",TZ,TZA,834
233,United Arab Emirates (the),AE,ARE,784
234,United Kingdom of Great Britain and Northern I...,GB,GBR,826
235,United States Minor Outlying Islands (the),UM,UMI,581
236,United States of America (the),US,USA,840


In [26]:
isoDf.loc[isoDf['Country'].str.contains('United States of America', case=False), 'Country'] = 'United States'
isoDf.loc[isoDf['Country'].str.contains('United Kingdom of Great Britain', case=False), 'Country'] = 'United Kingdom'
isoDf.loc[isoDf['Country'].str.contains('United Arab Emirates', case=False), 'Country'] = 'United Arab Emirates'
isoDf[isoDf['Country'].str.contains('United', case=False)]

,Country,iso_a2,iso_a3,iso_n
219,"Tanzania, United Republic of",TZ,TZA,834
233,United Arab Emirates,AE,ARE,784
234,United Kingdom,GB,GBR,826
235,United States Minor Outlying Islands (the),UM,UMI,581
236,United States,US,USA,840


In [27]:
# Czech Republic
isoDf[isoDf['Country'].str.contains('Czech', case=False)]

,Country,iso_a2,iso_a3,iso_n
59,Czechia,CZ,CZE,203


In [28]:
isoDf.loc[isoDf['Country'].str.contains('Czech', case=False), 'Country'] = 'Czech Republic'
isoDf[isoDf['Country'].str.contains('Czech', case=False)]

,Country,iso_a2,iso_a3,iso_n
59,Czech Republic,CZ,CZE,203


In [29]:
# South Korea
isoDf[isoDf['Country'].str.contains('Korea', case=False)]

,Country,iso_a2,iso_a3,iso_n
118,Korea (the Democratic People's Republic of),KP,PRK,408
119,Korea (the Republic of),KR,KOR,410


In [30]:
isoDf.loc[isoDf['Country']=='Korea (the Republic of)', 'Country'] = 'South Korea'
isoDf[isoDf['Country'].str.contains('Korea', case=False)]

,Country,iso_a2,iso_a3,iso_n
118,Korea (the Democratic People's Republic of),KP,PRK,408
119,South Korea,KR,KOR,410


In [31]:
# Taiwan
isoDf[isoDf['Country'].str.contains('Taiwan', case=False)]

,Country,iso_a2,iso_a3,iso_n
217,Taiwan (Province of China),TW,TWN,158


In [32]:
isoDf.loc[isoDf['Country']=='Taiwan (Province of China)', 'Country'] = 'Taiwan'
isoDf[isoDf['Country'].str.contains('Taiwan', case=False)]

,Country,iso_a2,iso_a3,iso_n
217,Taiwan,TW,TWN,158


In [33]:
# Bosnia And Herzegovina
isoDf[isoDf['Country'].str.contains('Bosnia', case=False)]

,Country,iso_a2,iso_a3,iso_n
28,Bosnia and Herzegovina,BA,BIH,70


In [34]:
isoDf.loc[isoDf['Country']=='Bosnia and Herzegovina', 'Country'] = 'Bosnia And Herzegovina'
isoDf[isoDf['Country'].str.contains('Bosnia', case=False)]

,Country,iso_a2,iso_a3,iso_n
28,Bosnia And Herzegovina,BA,BIH,70


In [35]:
# Macedonia
isoDf[isoDf['Country'].str.contains('Macedonia', case=False)]

,Country,iso_a2,iso_a3,iso_n
132,Macedonia (the former Yugoslav Republic of),MK,MKD,807


In [36]:
isoDf.loc[isoDf['Country']=='Macedonia (the former Yugoslav Republic of)', 'Country'] = 'Macedonia'
isoDf[isoDf['Country'].str.contains('Macedonia', case=False)]

,Country,iso_a2,iso_a3,iso_n
132,Macedonia,MK,MKD,807


In [37]:
# Russia
isoDf[isoDf['Country'].str.contains('Russia', case=False)]

,Country,iso_a2,iso_a3,iso_n
183,Russian Federation (the),RU,RUS,643


In [38]:
isoDf.loc[isoDf['Country']=='Russian Federation (the)', 'Country'] = 'Russia'
isoDf[isoDf['Country'].str.contains('Russia', case=False)]

,Country,iso_a2,iso_a3,iso_n
183,Russia,RU,RUS,643


In [39]:
# Philippines
isoDf[isoDf['Country'].str.contains('Philippines', case=False)]

,Country,iso_a2,iso_a3,iso_n
175,Philippines (the),PH,PHL,608


In [40]:
isoDf.loc[isoDf['Country']=='Philippines (the)', 'Country'] = 'Philippines'
isoDf[isoDf['Country'].str.contains('Philippines', case=False)]

,Country,iso_a2,iso_a3,iso_n
175,Philippines,PH,PHL,608


In [41]:
# Vietnam
isoDf[isoDf['Country'].str.contains('Viet', case=False)]

,Country,iso_a2,iso_a3,iso_n
241,Viet Nam,VN,VNM,704


In [42]:
isoDf.loc[isoDf['Country']=='Viet Nam', 'Country'] = 'Vietnam'
isoDf[isoDf['Country'].str.contains('Vietnam', case=False)]

,Country,iso_a2,iso_a3,iso_n
241,Vietnam,VN,VNM,704


In [43]:
# Iran
isoDf[isoDf['Country'].str.contains('Iran', case=False)]

,Country,iso_a2,iso_a3,iso_n
105,Iran (Islamic Republic of),IR,IRN,364


In [44]:
isoDf.loc[isoDf['Country']=='Iran (Islamic Republic of)', 'Country'] = 'Iran'
isoDf[isoDf['Country'].str.contains('Iran', case=False)]

,Country,iso_a2,iso_a3,iso_n
105,Iran,IR,IRN,364


In [45]:
# Venezula
isoDf[isoDf['Country'].str.contains('Vene', case=False)]

,Country,iso_a2,iso_a3,iso_n
240,Venezuela (Bolivarian Republic of),VE,VEN,862


In [46]:
isoDf.loc[isoDf['Country']=='Venezuela (Bolivarian Republic of)', 'Country'] = 'Venezuela'
isoDf[isoDf['Country'].str.contains('Venezuela', case=False)]

,Country,iso_a2,iso_a3,iso_n
240,Venezuela,VE,VEN,862


In [47]:
# cont...cleaning
countryDf[countryDf.iso_a2.isnull()].Country

3       United Arab Emirates
10               Netherlands
11             United States
17            United Kingdom
22            Czech Republic
37    Bosnia And Herzegovina
45               Philippines
46                    Russia
49                      Iran
54               South Korea
55                    Taiwan
66                 Venezuela
77        Dominican Republic
78                   Moldova
79                 Macedonia
81                   Bolivia
85                   Vietnam
Name: Country, dtype: object

In [48]:
# Dominican Republic
isoDf[isoDf['Country'].str.contains('Dom', case=False)]

,Country,iso_a2,iso_a3,iso_n
62,Dominica,DM,DMA,212
63,Dominican Republic (the),DO,DOM,214
234,United Kingdom,GB,GBR,826


In [49]:
isoDf.loc[isoDf['Country']=='Dominican Republic (the)', 'Country'] = 'Dominican Republic'
isoDf[isoDf['Country'].str.contains('Dominican Republic', case=False)]

,Country,iso_a2,iso_a3,iso_n
63,Dominican Republic,DO,DOM,214


In [50]:
# Moldova
isoDf[isoDf['Country'].str.contains('Mol', case=False)]

,Country,iso_a2,iso_a3,iso_n
146,Moldova (the Republic of),MD,MDA,498


In [51]:
isoDf.loc[isoDf['Country']=='Moldova (the Republic of)', 'Country'] = 'Moldova'
isoDf[isoDf['Country'].str.contains('Mol', case=False)]

,Country,iso_a2,iso_a3,iso_n
146,Moldova,MD,MDA,498


In [52]:
# Bolivia
isoDf[isoDf['Country'].str.contains('Bol', case=False)]

,Country,iso_a2,iso_a3,iso_n
26,Bolivia (Plurinational State of),BO,BOL,68


In [53]:
isoDf.loc[isoDf['Country']=='Bolivia (Plurinational State of)', 'Country'] = 'Bolivia'
isoDf[isoDf['Country'].str.contains('Bol', case=False)]

,Country,iso_a2,iso_a3,iso_n
26,Bolivia,BO,BOL,68


In [54]:
# cont...cleaning
countryDf[countryDf.iso_a2.isnull()].Country

3       United Arab Emirates
10               Netherlands
11             United States
17            United Kingdom
22            Czech Republic
37    Bosnia And Herzegovina
45               Philippines
46                    Russia
49                      Iran
54               South Korea
55                    Taiwan
66                 Venezuela
77        Dominican Republic
78                   Moldova
79                 Macedonia
81                   Bolivia
85                   Vietnam
Name: Country, dtype: object

In [55]:
# With Clean Data: Left Join Numbeo and ISO by Country
countryDf = merge_final.merge(isoDf, on='Country', how='left')
countryDf.head()

,Country,QII_2012,QII_2013,QII_2014,QII_2015,QII_2016,QII_2017,QII_2019,QII_2018,iso_a2,iso_a3,iso_n
0,Switzerland,194.11,215.71,206.23,222.94,208.54,173.54,195.93,190.81,CH,CHE,756
1,Germany,184.42,204.84,192.69,195.94,199.70,189.74,187.05,190.04,DE,DEU,276
2,Norway,183.43,173.86,168.41,175.09,188.90,165.93,181.86,176.27,NO,NOR,578
3,United Arab Emirates,177.07,186.01,173.27,153.68,144.09,139.27,167.81,163.54,AE,ARE,784
4,New Zealand,174.28,163.17,168.56,175.51,201.06,184.74,185.58,182.40,NZ,NZL,554


In [56]:
# Check for null columns
null_columns = countryDf.columns[countryDf.isnull().any()]
countryDf[null_columns].isnull().sum()

QII_2012    36
QII_2013    20
QII_2014    19
QII_2015     1
QII_2016    31
QII_2017    20
QII_2019    16
QII_2018    27
dtype: int64

In [57]:
# Final table 2019
countryDf.tail()

,Country,QII_2012,QII_2013,QII_2014,QII_2015,QII_2016,QII_2017,QII_2019,QII_2018,iso_a2,iso_a3,iso_n
82,Nigeria,NaN,NaN,NaN,2.99,NaN,NaN,NaN,NaN,NG,NGA,566
83,Cambodia,NaN,NaN,NaN,-4.40,NaN,NaN,NaN,NaN,KH,KHM,116
84,Bangladesh,NaN,NaN,NaN,-5.36,NaN,NaN,NaN,NaN,BD,BGD,50
85,Vietnam,NaN,NaN,NaN,-19.52,NaN,36.12,88.82,81.09,VN,VNM,704
86,Iceland,NaN,NaN,NaN,NaN,NaN,NaN,187.79,NaN,IS,ISL,352


In [58]:
# To CSV
countryDf.to_csv('db/QII.csv')

In [59]:
QII = pd.read_csv('db/QII.csv')
table_QII = QII.drop(columns=['Unnamed: 0'])
table_QII.head()

,Country,QII_2012,QII_2013,QII_2014,QII_2015,QII_2016,QII_2017,QII_2019,QII_2018,iso_a2,iso_a3,iso_n
0,Switzerland,194.11,215.71,206.23,222.94,208.54,173.54,195.93,190.81,CH,CHE,756
1,Germany,184.42,204.84,192.69,195.94,199.70,189.74,187.05,190.04,DE,DEU,276
2,Norway,183.43,173.86,168.41,175.09,188.90,165.93,181.86,176.27,NO,NOR,578
3,United Arab Emirates,177.07,186.01,173.27,153.68,144.09,139.27,167.81,163.54,AE,ARE,784
4,New Zealand,174.28,163.17,168.56,175.51,201.06,184.74,185.58,182.40,NZ,NZL,554


In [60]:
table_GDP =pd.read_csv('db/GDP.csv')
table_GDP = table_GDP.rename(columns={
    '2012':'GDP_2012',
    '2013':'GDP_2013',
    '2014':'GDP_2014',
    '2015':'GDP_2015',
    '2016':'GDP_2016',
    '2017':'GDP_2017',
    '2018':'GDP_2018',
    '2019':'GDP_2019'
}).copy()
table_GDP.head()

,Country,GDP_2012,GDP_2013,GDP_2014,GDP_2015,GDP_2016,GDP_2017
0,Afghanistan,661.087,635.635,629.332,594.513,560.601,569.531
1,Albania,4248.814,4422.674,4575.416,3944.876,4126.557,4539.247
2,Algeria,5574.507,5477.055,5466.329,4153.322,3921.291,4016.087
3,Angola,5245.023,5436.516,5625.736,4354.921,3676.826,4303.696
4,Antigua and Barbuda,14006.82,13646.242,14436.522,15219.198,16225.279,16619.974


In [61]:
Df1 = table_QII.merge(table_GDP, on='Country', how='left')


In [62]:
null_columns = Df1.columns[Df1.isnull().any()]
Df1[null_columns].isnull().sum()
Df1[Df1.iso_a2.isnull()].Country

Series([], Name: Country, dtype: object)

In [63]:
#Fixing South Korea

table_GDP.loc[table_GDP['Country']=='Korea, Republic of', 'Country'] = 'South Korea'
table_GDP[table_GDP['Country'].str.contains('South Korea', case=False)]

,Country,GDP_2012,GDP_2013,GDP_2014,GDP_2015,GDP_2016,GDP_2017
89,South Korea,24358.78,25890.016,27811.367,27105.078,27608.248,29749.796


In [64]:
#Fixing Slovakia

table_GDP.loc[table_GDP['Country']=='Slovak Republic', 'Country'] = 'Slovakia'
table_GDP[table_GDP['Country'].str.contains('Slovakia', case=False)]

,Country,GDP_2012,GDP_2013,GDP_2014,GDP_2015,GDP_2016,GDP_2017
155,Slovakia,17294.62,18205.908,18668.684,16197.783,16564.909,17627.111


In [65]:
#Fixing Slovakia

table_GDP.loc[table_GDP['Country']=='Slovak Republic', 'Country'] = 'Slovakia'
table_GDP[table_GDP['Country'].str.contains('Slovakia', case=False)]

,Country,GDP_2012,GDP_2013,GDP_2014,GDP_2015,GDP_2016,GDP_2017
155,Slovakia,17294.62,18205.908,18668.684,16197.783,16564.909,17627.111


In [66]:
#Fixing Bosnia and Herzegovina

table_GDP.loc[table_GDP['Country']=='Bosnia and Herzegovina', 'Country'] = 'Bosnia And Herzegovina'
table_GDP[table_GDP['Country'].str.contains('Bosnia And Herzegovina', case=False)]

,Country,GDP_2012,GDP_2013,GDP_2014,GDP_2015,GDP_2016,GDP_2017
21,Bosnia And Herzegovina,4716.643,5036.032,5194.014,4584.264,4808.299,5180.778


In [67]:

#Fixing Taiwan

table_GDP.loc[table_GDP['Country']=='Taiwan Province of China', 'Country'] = 'Taiwan'
table_GDP[table_GDP['Country'].str.contains('Taiwan', case=False)]

,Country,GDP_2012,GDP_2013,GDP_2014,GDP_2015,GDP_2016,GDP_2017
168,Taiwan,21269.614,21887.992,22638.917,22373.564,22572.702,24389.677


In [68]:
#Fixing Macedonia (note - North Macedonia has a space at the end in the CSV file)

table_GDP.loc[table_GDP['Country']=='North Macedonia ', 'Country'] = 'Macedonia'
table_GDP[table_GDP['Country'].str.contains('Macedonia', case=False)]

,Country,GDP_2012,GDP_2013,GDP_2014,GDP_2015,GDP_2016,GDP_2017
127,Macedonia,4728.267,5239.722,5498.606,4860.421,5153.016,5448.428


In [69]:

# Fixing Hong Kong

table_GDP.loc[table_GDP['Country']=='Hong Kong SAR', 'Country'] = 'Hong Kong'
table_GDP[table_GDP['Country'].str.contains('Hong Kong', case=False)]


,Country,GDP_2012,GDP_2013,GDP_2014,GDP_2015,GDP_2016,GDP_2017
73,Hong Kong,36619.811,38230.066,40182.28,42321.712,43496.299,46091.216


In [70]:
#Fixing China

table_GDP.loc[table_GDP['Country']=="China, People's Republic of", 'Country'] = 'China'
table_GDP[table_GDP['Country'].str.contains('China', case=False)]

,Country,GDP_2012,GDP_2013,GDP_2014,GDP_2015,GDP_2016,GDP_2017
35,China,6329.464,7080.828,7701.691,8166.756,8115.828,8677.4


In [71]:
#Fixing Russia

table_GDP.loc[table_GDP['Country']=='Russian Federation', 'Country'] = 'Russia'
table_GDP[table_GDP['Country'].str.contains('Russia', case=False)]

,Country,GDP_2012,GDP_2013,GDP_2014,GDP_2015,GDP_2016,GDP_2017
142,Russia,15358.085,15942.141,14305.57,9477.559,8909.547,10961.992


In [72]:
Df1 = table_QII.merge(table_GDP, on='Country', how='left')

null_columns = Df1.columns[Df1.isnull().any()]
Df1[null_columns].isnull().sum()
Df1[Df1.iso_a2.isnull()].Country
Df1.head()


,Country,QII_2012,QII_2013,QII_2014,QII_2015,QII_2016,QII_2017,QII_2019,QII_2018,iso_a2,iso_a3,iso_n,GDP_2012,GDP_2013,GDP_2014,GDP_2015,GDP_2016,GDP_2017
0,Switzerland,194.11,215.71,206.23,222.94,208.54,173.54,195.93,190.81,CH,CHE,756,83958.545,85675.755,87161.617,82510.452,80490.76,80642.579
1,Germany,184.42,204.84,192.69,195.94,199.70,189.74,187.05,190.04,DE,DEU,276,44089.275,46544.947,48218.87,41415.169,42460.686,44770.578
2,Norway,183.43,173.86,168.41,175.09,188.90,165.93,181.86,176.27,NO,NOR,578,101273.375,102722,96837.607,74280.697,70703.165,75513.639
3,United Arab Emirates,177.07,186.01,173.27,153.68,144.09,139.27,167.81,163.54,AE,ARE,784,42723.581,43197.43,43340.018,37380.571,36226.239,37732.656
4,New Zealand,174.28,163.17,168.56,175.51,201.06,184.74,185.58,182.40,NZ,NZL,554,39553.735,41775.712,43857.784,37741.881,38982.796,41350.257


In [73]:
Df1.to_csv('db/QII_GPD.csv', index=False)

In [74]:
Dfnew=pd.read_csv('db/QII_GPD.csv')
Dfnew["QII_2012"][52]=160.39
Dfnew["QII_2012"][54]=132.55

Dfnew=Dfnew.dropna()
Dfnew.iloc[:,-6:]=round(Dfnew.iloc[:,-6:],2)
Dfnew

C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Country,QII_2012,QII_2013,QII_2014,QII_2015,QII_2016,QII_2017,QII_2019,QII_2018,iso_a2,iso_a3,iso_n,GDP_2012,GDP_2013,GDP_2014,GDP_2015,GDP_2016,GDP_2017
0,Switzerland,194.11,215.71,206.23,222.94,208.54,173.54,195.93,190.81,CH,CHE,756,83958.54,85675.76,87161.62,82510.45,80490.76,80642.58
1,Germany,184.42,204.84,192.69,195.94,199.70,189.74,187.05,190.04,DE,DEU,276,44089.28,46544.95,48218.87,41415.17,42460.69,44770.58
2,Norway,183.43,173.86,168.41,175.09,188.90,165.93,181.86,176.27,NO,NOR,578,101273.38,102722.00,96837.61,74280.70,70703.16,75513.64
3,United Arab Emirates,177.07,186.01,173.27,153.68,144.09,139.27,167.81,163.54,AE,ARE,784,42723.58,43197.43,43340.02,37380.57,36226.24,37732.66
4,New Zealand,174.28,163.17,168.56,175.51,201.06,184.74,185.58,182.40,NZ,NZL,554,39553.74,41775.71,43857.78,37741.88,38982.80,41350.26
5,Sweden,171.72,191.36,180.92,193.86,185.81,172.74,178.67,176.81,SE,SWE,752,56978.61,60069.35,58930.15,50565.10,51245.36,52924.36
6,Canada,164.99,186.03,178.29,177.63,177.23,167.18,170.32,173.90,CA,CAN,124,52753.35,52726.05,50958.06,43615.80,42446.55,45223.97
7,Denmark,163.12,182.29,178.55,190.18,206.49,184.92,198.57,197.75,DK,DNK,208,58623.41,61325.58,62729.50,53478.50,54665.22,57380.20
8,Australia,162.03,165.80,175.98,180.81,198.79,176.54,191.13,188.70,AU,AUS,36,68436.22,65170.85,61652.28,51493.96,51982.83,55957.72
9,Austria,159.89,167.39,171.82,182.62,192.40,190.37,191.05,190.22,AT,AUT,40,48615.82,50748.09,51814.37,44266.41,45105.75,47383.87


In [75]:
cols = [9,11]
Dfnew1=Dfnew.copy()
Dfnew1.drop(Dfnew.columns[cols],axis=1,inplace=True)
# dfee["2017"] = pd.to_numeric(dfee["2017"])
# dfee["2017comb"]=dfee["2017"]*dfee["QII"]
Dfnew2=Dfnew1

In [76]:
Dfnew2=Dfnew1.rename(columns={"iso_a3":"ISO_A3"})
 
Dfnew2.tail()


,Country,QII_2012,QII_2013,QII_2014,QII_2015,QII_2016,QII_2017,QII_2019,QII_2018,ISO_A3,GDP_2012,GDP_2013,GDP_2014,GDP_2015,GDP_2016,GDP_2017
48,Egypt,-22.17,7.59,5.41,-7.07,88.79,91.81,83.98,84.42,EGY,3383.11,3400.32,3524.42,3731.18,3686.07,2495.02
49,Iran,-35.23,-6.96,-11.08,1.07,96.93,97.17,87.02,92.43,IRN,5118.47,5152.04,5395.79,4723.48,5026.65,5289.80
50,China,-49.55,31.50,30.30,15.99,99.03,90.95,97.92,99.43,CHN,6329.46,7080.83,7701.69,8166.76,8115.83,8677.40
52,Finland,160.39,167.21,178.88,190.25,184.01,182.93,194.01,195.30,FIN,47553.40,49766.13,50087.85,42505.91,43582.43,45937.75
54,South Korea,132.55,135.62,117.90,120.03,170.29,162.49,149.53,155.38,KOR,24358.78,25890.02,27811.37,27105.08,27608.25,29749.80


In [77]:
dfk=pd.read_json("db/countries.geojson")
dfk=dfk.drop(["type"],axis =1)
dfk.head()

,features
0,"{'type': 'Feature', 'properties': {'ADMIN': 'A..."
1,"{'type': 'Feature', 'properties': {'ADMIN': 'A..."
2,"{'type': 'Feature', 'properties': {'ADMIN': 'A..."
3,"{'type': 'Feature', 'properties': {'ADMIN': 'A..."
4,"{'type': 'Feature', 'properties': {'ADMIN': 'A..."


In [78]:
dict={"type":"FeatureCollection","features":[]}
for x in range(len(dfk)):
    dict["features"].append(dfk.features[x])
for x in range(len(dfk)-1,-1,-1):
    if dict["features"][x]["properties"]["ISO_A3"] not in Dfnew2["ISO_A3"].values:
        dict["features"].pop(x)
len(dict["features"])

53

In [79]:
cdf=Dfnew2.sort_values(by=['ISO_A3'])
cdf=cdf.reset_index(drop=True)
cdf

,Country,QII_2012,QII_2013,QII_2014,QII_2015,QII_2016,QII_2017,QII_2019,QII_2018,ISO_A3,GDP_2012,GDP_2013,GDP_2014,GDP_2015,GDP_2016,GDP_2017
0,United Arab Emirates,177.07,186.01,173.27,153.68,144.09,139.27,167.81,163.54,ARE,42723.58,43197.43,43340.02,37380.57,36226.24,37732.66
1,Argentina,32.58,72.53,81.12,77.01,138.48,139.59,122.49,131.85,ARG,13889.79,14488.83,13208.83,14895.32,12772.87,14588.01
2,Australia,162.03,165.80,175.98,180.81,198.79,176.54,191.13,188.70,AUS,68436.22,65170.85,61652.28,51493.96,51982.83,55957.72
3,Austria,159.89,167.39,171.82,182.62,192.40,190.37,191.05,190.22,AUT,48615.82,50748.09,51814.37,44266.41,45105.75,47383.87
4,Belgium,100.68,146.73,134.71,135.95,162.29,160.52,162.09,164.00,BEL,44977.07,46784.99,47550.18,40585.20,41545.96,43672.41
5,Bulgaria,30.77,53.71,62.60,68.30,141.61,138.20,130.59,129.69,BGR,7399.39,7667.59,7888.56,7017.41,7496.08,8274.48
6,Bosnia And Herzegovina,28.04,71.60,74.60,73.45,146.70,139.14,124.51,129.33,BIH,4716.64,5036.03,5194.01,4584.26,4808.30,5180.78
7,Brazil,10.81,48.21,38.76,29.84,101.02,96.15,100.33,100.13,BRA,12426.69,12357.57,12176.25,8846.38,8752.33,9928.23
8,Canada,164.99,186.03,178.29,177.63,177.23,167.18,170.32,173.90,CAN,52753.35,52726.05,50958.06,43615.80,42446.55,45223.97
9,Switzerland,194.11,215.71,206.23,222.94,208.54,173.54,195.93,190.81,CHE,83958.54,85675.76,87161.62,82510.45,80490.76,80642.58


In [80]:
for x in range(len(dict["features"])):
    temp=cdf.loc[x]
    tempdf=pd.DataFrame(temp).T
    tempdict=tempdf.to_dict("records")
    dict["features"][x]["properties"].update(tempdict[0])
dict["features"][0]["properties"]

{'ADMIN': 'United Arab Emirates',
 'ISO_A3': 'ARE',
 'Country': 'United Arab Emirates',
 'QII_2012': 177.07,
 'QII_2013': 186.01,
 'QII_2014': 173.27,
 'QII_2015': 153.68,
 'QII_2016': 144.09,
 'QII_2017': 139.27,
 'QII_2019': 167.81,
 'QII_2018': 163.54,
 'GDP_2012': 42723.58,
 'GDP_2013': 43197.43,
 'GDP_2014': 43340.02,
 'GDP_2015': 37380.57,
 'GDP_2016': 36226.24,
 'GDP_2017': 37732.66}

In [81]:
with open('db/resultnew.json', 'w') as fp:
    json.dump(dict, fp)

In [82]:

# The default port used by MongoDB is 27017
# https://docs.mongodb.com/manual/reference/default-mongodb-port/
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [83]:
db = client.Project2_data
collection = db.GEOJSON
collection.remove()
collection.insert_one(dict)

C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.
  This is separate from the ipykernel package so we can avoid doing imports until
